In [1]:
import pandas as pd

finetuned_dirname = "roberta-finetuned-phemernr2"
transformer_name = "roberta-base"

data = pd.read_csv("../../data/phemernr2_dataset_with_tvt.csv", lineterminator="\n")
data = data[['tweet_text', 'tvt2', 'label2']]
data.columns = ['tweet_text', 'tvt2', 'label']
print(data.shape)
data.head()

(6425, 3)


,tweet_text,tvt2,label
0,The East London Mosque would like to offer its...,testting,non-rumours
1,BREAKING - A Germanwings Airbus A320 plane rep...,training,true
2,Reports that two of the dead in the #CharlieHe...,validation,true
3,After #Putin disappeared Russian TV no longer ...,training,non-rumours
4,Saw #Ferguson for myself. #justiceformichaelbr...,training,non-rumours


In [2]:
data['label'].value_counts()

non-rumours    4023
true           1067
unverified      697
false           638
Name: label, dtype: int64

In [3]:
combined_data = data

In [4]:
import torch

class CustomTextDataset(torch.utils.data.dataset.Dataset):

    def __init__(self, texts, labels):
        self.labels = labels
        self.texts = texts
        self.attention_mask = None
        self.input_ids = None
        self.token_type_ids = None

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        sample = {
            "text": self.texts[idx],
            "label": self.labels[idx],
            "attention_mask": self.attention_mask[idx] if self.attention_mask else None,
            "input_ids": self.input_ids[idx] if self.input_ids else None,
#             "token_type_ids": self.token_type_ids[idx] if self.token_type_ids else None
        }
        return sample
    
    def tokenize(self, tokenizer):
        self.attention_mask = []
        self.input_ids = []
        self.token_type_ids = []

        for text in self.texts:
            token = tokenizer(text, padding="max_length", truncation=True)
            
            self.attention_mask.append(token['attention_mask'])
            self.input_ids.append(token['input_ids'])
#             self.token_type_ids.append(token['token_type_ids'])

In [5]:
labels = []

labels_str = combined_data['label'].unique().tolist()
for i, d in combined_data.iterrows():
    lab = labels_str.index(d['label'])
    labels.append(lab)
    
print(len(labels))
labels[:10]

6425


[0, 1, 1, 0, 0, 0, 0, 2, 0, 0]

In [6]:
train_dataset = CustomTextDataset(
    [d['tweet_text'] for i, d in combined_data.iterrows() if d['tvt2'] == 'training'],
    [labels[i] for i, d in combined_data.iterrows() if d['tvt2'] == 'training'])
test_dataset = CustomTextDataset(
    [d['tweet_text'] for i, d in combined_data.iterrows() if d['tvt2'] == 'validation'],
    [labels[i] for i, d in combined_data.iterrows() if d['tvt2'] == 'validation'])
train_dataset[0]

{'text': 'BREAKING - A Germanwings Airbus A320 plane reportedly crashed in the region of Digne (French Alps) #Flightradar24 - French TV #itele',
 'label': 1,
 'attention_mask': None,
 'input_ids': None}

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(transformer_name)

In [8]:
# inputs = tokenizer(["you're stuck in a timewrap from 2004 though", "summa lumma dumma lumma"], padding="max_length", truncation=True)
# for k,v in inputs.items():
#     print(k)

In [9]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_dataset.tokenize(tokenizer)
test_dataset.tokenize(tokenizer)

In [10]:
print(len(train_dataset))
print(len(test_dataset))

4323
1418


### Fine Tuning

In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(transformer_name, output_hidden_states=False, num_labels=4)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [12]:
from transformers import TrainingArguments

epochs = 10
batch_size = 8
save_steps = (round((len(train_dataset)/batch_size) + 0.5)) * epochs
# save_steps = 1_000_000

training_args = TrainingArguments(
    output_dir=f"../../data/models/{finetuned_dirname}",
    num_train_epochs=epochs,
    save_steps=save_steps,
    logging_steps=300,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size
)

print(f"Save Steps : {save_steps}")

Save Steps : 5410


In [13]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [15]:
import time

start = time.time()

trainer.train()

print(f"Execution Time : {round(time.time() - start)} seconds")

***** Running training *****
  Num examples = 4323
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5410


Step,Training Loss
300,0.904100
600,0.698800
900,0.601200
1200,0.532100
1500,0.489800
1800,0.370500
2100,0.360400
2400,0.283000
2700,0.246000
3000,0.178600


Saving model checkpoint to ../../data/models/roberta-finetuned-phemernr2/checkpoint-5410
Configuration saved in ../../data/models/roberta-finetuned-phemernr2/checkpoint-5410/config.json
Model weights saved in ../../data/models/roberta-finetuned-phemernr2/checkpoint-5410/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




Execution Time : 1736 seconds


In [16]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1418
  Batch size = 8


{'eval_loss': 1.179558515548706,
 'eval_accuracy': 0.8455571227080395,
 'eval_runtime': 16.9848,
 'eval_samples_per_second': 83.486,
 'eval_steps_per_second': 10.48,
 'epoch': 10.0}